<a href="https://colab.research.google.com/github/mshsu/probasets/blob/main/scrape/crime_scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import pandas as pd
import requests
import time
from random import sample

In [5]:
st_abbr = pd.read_csv("https://www2.census.gov/geo/docs/reference/state.txt", sep='|')['STUSAB']

In [25]:
url = "https://api.usa.gov/crime/fbi/cde/"

In [6]:
keyfile = open('account.info', 'r')
key = keyfile.read()

In [ ]:
agencies = []
for state in st_abbr:
  try:
    print("Fetching " + state + "...", end = "")
    response = requests.get(url + "agency/byStateAbbr/" + state + "?API_KEY=" + key)
    raw = response.json()
    oris = []
    for agcy in raw:
      oris.append(agcy["ori"])
    print(" OK")
    agencies = agencies + oris
    time.sleep(0.25)
  except:
    print(' NO DATA')

In [ ]:
crimes = ['aggravated-assault', 'violent-crime', 'robbery', 'arson',
          'human-trafficing', 'rape-legacy', 'homicide', 'burglary',
          'motor-vehicle-theft', 'larceny', 'rape', 'property-crime']
stats = []

for ori in sample(agencies, 2000):
  estimates = dict.fromkeys(['ori'] + crimes)
  estimates['ori'] = ori
  for c in crimes:
    response = requests.get(url + "summarized/agency/" + ori + "/" 
                            + c + "?from=2021&to=2021&API_KEY=" + key)
    raw = response.json()
    if len(raw) > 0:
      estimates[c] = raw[0]['actual']
    time.sleep(0.01)
  stats.append(estimates)

In [ ]:
pd.DataFrame(stats)

In [ ]:
pd.DataFrame(stats).to_csv('Crime.csv', index = False)